# Llama3SP - Model Inspection Notebook

#### Hello!  Welcome to the model inspection notebook, we provides the scripts for model loading and model inference on testing data of model training process for all of the models mentioned in our experiments in the paper. 

##### Attention!!!
##### Before interacting with this notebook, you may want to install a few dependencies [HERE](#dependencies).
##### Also, make sure to run the [Static Methods](#static-method) cell, then you are good to go
#### The models are categorized by the experiment scenario, please follow the link as follows to reach the specific section

### 1. [Within Project Models](#within_project)
### 2. [Cross Project Models](#cross_project)

<a id='dependencies'></a>
## Dependencies Installation
#### run the cell below to install the dependencies

In [ ]:
%!pip install transformers
%!pip install peft
%!pip install torch
%!pip install tokenizers
%!pip install captum

<a id='static-method'></a>
## Static Methods

In [1]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from Llama3SP import LlamaForSequenceClassification as Llama3SP
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    XLNetTokenizer,
    BertTokenizer,
    BitsAndBytesConfig,
)
from peft import (
    PeftModel,
    PeftConfig,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
)
from tokenizers import Tokenizer
from pathlib import Path

import torch
import pandas as pd
import numpy as np
import os
import gc
import shutil
import re

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Tokenizer Mapping
TOKENIZER_MAPPING = {"#0": "llama3", "#00": "llama3", "#000": "llama3",
                     "#2": "sp_word_level", "#22": "sp_word_level", "#222": "sp_word_level",
                     "#6": "wordpiece_sp", "#66": "wordpiece_sp", "#666": "wordpiece_sp",
                     "#7": "sentencepiece_sp", "#77": "sentencepiece_sp", "#777": "sentencepiece_sp"}

# pad token ID mapping
PAD_TOKEN_ID_MAPPING = {"llama3": 128001, "sp_word_level": 3, "wordpiece_sp": 0, "sentencepiece_sp": 0}
# static global vars
global DYNAMIC_BATCH, DEVICE
DYNAMIC_BATCH = True
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    # set up to release cache memory when possible
    torch.cuda.empty_cache()
    # set up more conservative memory limits  
    torch.cuda.set_per_process_memory_fraction(0.8)  # Use only 80% of GPU memory
SEQUENCE_LEN = 20
# dynamic global vars
global PAD_TOKEN_ID, BATCH_SIZE_RATIO, BATCH_SIZE, TOKENIZER, WITHIN_PROJECT, TEXT, KEY, TOK, MODEL, PROJECT_ID
PAD_TOKEN_ID = None
BATCH_SIZE_RATIO = None
BATCH_SIZE = None
TOKENIZER = None
WITHIN_PROJECT = None
TEXT = None
KEY = None
TOK = None
MODEL = None
PROJECT_ID = None


def tokenization(text_list, path):
    global TOKENIZER, SEQUENCE_LEN, MODEL, TOK, PAD_TOKEN_ID
    
    if TOKENIZER == 'llama3':
        print('using llama3 tokenizer!')
        tokenizer = AutoTokenizer.from_pretrained(path)

        tokenizer.pad_token = tokenizer.eos_token
        MODEL.config.pad_token_id = MODEL.config.eos_token_id 
        
        # ensure that no sequence exceeds SEQUENCE_LEN
        encoded_dict = tokenizer.batch_encode_plus(
            text_list,
            max_length=SEQUENCE_LEN,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True,
            return_token_type_ids=False
        )
        
        # Verify dimensions
        if encoded_dict['input_ids'].shape[1] > SEQUENCE_LEN:
            print(f"Warning: Truncating sequences to {SEQUENCE_LEN} tokens")
            encoded_dict['input_ids'] = encoded_dict['input_ids'][:, :SEQUENCE_LEN]
            if 'attention_mask' in encoded_dict:
                encoded_dict['attention_mask'] = encoded_dict['attention_mask'][:, :SEQUENCE_LEN]
        
        TOK = tokenizer
        return encoded_dict
    elif TOKENIZER == 'sp_word_level':
        print('using word-level tokenizer!')
        tokenizer = Tokenizer.from_pretrained(path)
        encoded_sentences = {'input_ids':[]}
        for sentence in text_list:
            encoded = tokenizer.encode(sentence)
            encoded = encoded.ids
            if len(encoded) > SEQUENCE_LEN:
                encoded = encoded[:SEQUENCE_LEN]
            elif len(encoded) < SEQUENCE_LEN:
                padding = SEQUENCE_LEN - len(encoded)
                for _ in range(padding):
                    encoded.append(3)
            encoded_sentences['input_ids'].append(encoded)
        tokenizer.pad_token_id = PAD_TOKEN_ID
        MODEL.config.pad_token_id = PAD_TOKEN_ID

        TOK = tokenizer
        return encoded_sentences
    elif TOKENIZER == 'sentencepiece_sp':
        print('using sentencepiece tokenizer!')
        tokenizer = XLNetTokenizer('all_tokenizers/sp_sentence_piece/spm_tokenizer.model', padding_side='right')

        tokenizer.pad_token_id = tokenizer.eos_token_id
        tokenizer.pad_token = tokenizer.eos_token
        # update some model configs
        # must use .cache = False as below or it crashes from my experience
        MODEL.config.pad_token_id = tokenizer.pad_token_id
        MODEL.config.use_cache = False
        MODEL.config.pretraining_tp = 1
        return tokenizer.batch_encode_plus(text_list, truncation=True, max_length=SEQUENCE_LEN, padding='max_length')
    elif TOKENIZER == 'wordpiece_sp':
        print('using wordpiece tokenizer!')
        tokenizer = BertTokenizer('all_tokenizers/sp_word_piece/vocab.txt')

        MODEL.config.pad_token_id = tokenizer.pad_token_id
        MODEL.config.use_cache = False
        MODEL.config.pretraining_tp = 1
        return tokenizer.batch_encode_plus(text_list, truncation=True, max_length=SEQUENCE_LEN, padding='max_length')


def prepare_dataframe(file_name):
    data = pd.read_csv(file_name)
    # some rows have no description, fill blank to avoid Null
    data = data.fillna(' ')
    d = {'text': data['title'], 'label': data['storypoint'], 'issuekey': data['issuekey']}
    return pd.DataFrame(data=d)


def prepare_dataloader(seq, y, sampler_type):
    global BATCH_SIZE
    tensor_dataset = TensorDataset(seq, y)
    if sampler_type == 'random':
        sampler = RandomSampler(tensor_dataset)
    elif sampler_type == 'sequential':
        sampler = SequentialSampler(tensor_dataset)
    dataloader = DataLoader(tensor_dataset, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader


def load_trained_model(model_id, project_name):
    global WITHIN_PROJECT, MODEL, PROJECT_ID

    local = False
    try:
        int(model_id[1:])
        local = False
    except:
        local = True

    if local:
        print("Loading model from local...")

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )
        lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],
            lora_dropout=0.1,
            bias='none',
            task_type='SEQ_CLS'
        )
        HF_MODEL_NAME = "meta-llama/Llama-3.2-1B"
        MODEL = AutoModelForSequenceClassification.from_pretrained(
            HF_MODEL_NAME,
            quantization_config=quantization_config,
            num_labels=1,
            torch_dtype=torch.float16,
            device_map='auto',
            low_cpu_mem_usage=True,
            pad_token_id=PAD_TOKEN_ID,
        )
        MODEL = prepare_model_for_kbit_training(MODEL)
        MODEL = get_peft_model(MODEL, lora_config)
        MODEL.gradient_checkpointing_enable()
        MODEL.enable_input_require_grads()

        state_dict = torch.load(model_id, map_location=DEVICE, weights_only=True)
        MODEL.load_state_dict(state_dict, strict=False)

    else:
        print("Loading model from Hugging Face...")

        if WITHIN_PROJECT:
            path = "DEVCamiloSepulveda/" + model_id[1:] + "-LLAMA3SP-" + project_name
        else:
            path = "DEVCamiloSepulveda/" + model_id[1:] + "-LLAMA3SP-" + project_name.split("_")[0] + "-" + project_name.split("_")[1]
        PROJECT_ID = path

        # Load the model configuration
        config = PeftConfig.from_pretrained(path)

        # Load the original base model
        base_model = AutoModelForSequenceClassification.from_pretrained(
            config.base_model_name_or_path,
            num_labels=1,
            torch_dtype=torch.float16,
            device_map='auto'
        )

        # Load the LoRA adapters
        MODEL = PeftModel.from_pretrained(base_model, path)
    
    return MODEL


def prepare_test_dataloader(file_name, model_id, project_name):
    global WITHIN_PROJECT, BATCH_SIZE, BATCH_SIZE_RATIO, TEXT, KEY, MODEL

    global WITHIN_PROJECT, MODEL
    if WITHIN_PROJECT:
        path = "DEVCamiloSepulveda/" + model_id[1:] + "-LLAMA3SP-" + project_name
    else:
        path = "DEVCamiloSepulveda/" + model_id[1:] + "-LLAMA3SP-" + project_name.split("_")[0] + "-" + project_name.split("_")[1]
    path = "meta-llama/Llama-3.2-1B"

    if WITHIN_PROJECT:
        # calculate the batch size
        df = prepare_dataframe(file_name)
        BATCH_SIZE = min(int(int(len(df['text'][:int(len(df)*0.6)])) * BATCH_SIZE_RATIO), 512)
        print("Batch Size: ", BATCH_SIZE)
        # prepare testing data
        test_text = df['text'][int(len(df)*0.8):]
        TEXT = test_text
        KEY = df['issuekey'][int(len(df)*0.8):]
        test_labels = df['label'][int(len(df)*0.8):]
        tokens_test = tokenization(test_text.tolist(), path)
        test_seq = torch.tensor(tokens_test['input_ids'])
        test_y = torch.tensor(test_labels.tolist()).type(torch.LongTensor)
        test_dataloader = prepare_dataloader(test_seq, test_y, sampler_type='sequential')
    else:
        # calculate the batch size based on training data
        df = prepare_dataframe('sp_dataset/marked_data/' + file_name[0] + '.csv')
        BATCH_SIZE = min(int(int(len(df['text'][:int(len(df)*0.6)])) * BATCH_SIZE_RATIO), 512)
        # prepare testing data
        df = prepare_dataframe('sp_dataset/marked_data/' + file_name[1] + '.csv')
        test_text = df['text']
        TEXT = test_text
        KEY = df['issuekey']
        test_labels = df['label']
        tokens_test = tokenization(test_text.tolist(), path)
        test_seq = torch.tensor(tokens_test['input_ids'])
        test_y = torch.tensor(test_labels.tolist()).type(torch.LongTensor)
        test_dataloader = prepare_dataloader(test_seq, test_y, sampler_type='sequential')         
    return test_dataloader


def calculate_mae(predictions, targets):
    return np.mean(np.abs(predictions - targets))


def do_inference(trained_model, test_dataloader):
    global TEXT, KEY
    global XAI
    predictions = []
    true_labels = []
    try:
        for batch in test_dataloader:
            batch = tuple(t.to(DEVICE) for t in batch)
            b_input_ids, b_labels = batch
            with torch.no_grad():
                logits = trained_model(b_input_ids)
            logits = logits['logits'].detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.append(logits)
            true_labels.append(label_ids)
        
        # Flatten predictions and true_labels for MAE calculation
        flat_predictions = np.concatenate(predictions, axis=0).flatten()
        flat_true_labels = np.concatenate(true_labels, axis=0).flatten()
        
        # Calculate MAE of the model (MAE_p)
        MAE_p = calculate_mae(flat_predictions, flat_true_labels)
        
        # Generate random predictions and calculate MAE of random predictions (MAE_Po)
        random_predictions = np.random.rand(1000, len(flat_true_labels))  # 1000 ejecuciones de predicciones aleatorias
        MAE_Po = np.mean([calculate_mae(pred, flat_true_labels) for pred in random_predictions])
        
        # Calculate Standard Accuracy (SA)
        SA = (1 - (MAE_p / MAE_Po)) * 100
        
        # Calculate Median Absolute Error (MdAE)
        MdAE = np.median(np.abs(flat_predictions - flat_true_labels))
        
        return MAE_p, MdAE, SA
        
    except Exception as e:
        print(f"Unexpected error in do_inference: {e}")
        import traceback
        traceback.print_exc()
        return None


def main(model_id, project_name):
    global WITHIN_PROJECT, BATCH_SIZE_RATIO, TOKENIZER, PAD_TOKEN_ID, MODEL
    # define tokenizer based on model ID
    try:
        TOKENIZER = TOKENIZER_MAPPING[model_id]
    except:
        TOKENIZER = 'llama3'
    PAD_TOKEN_ID = PAD_TOKEN_ID_MAPPING[TOKENIZER]

    if len(project_name.split('_')) == 1:
        WITHIN_PROJECT = True
        BATCH_SIZE_RATIO = 0.3
        print('within project inference using model ' + model_id + ' for project ' + project_name)
        file_name = 'sp_dataset/marked_data/' + project_name + '.csv'
    else:
        WITHIN_PROJECT = False
        BATCH_SIZE_RATIO = 0.4
        training_project = project_name.split('_')[0]
        testing_project = project_name.split('_')[1]
        print('cross project inference using model ' + model_id + ' trained on ' + training_project 
              + ' for project ' + testing_project)
        file_name = (training_project, testing_project)
    trained_model = load_trained_model(model_id, project_name)
    
    trained_model.to(DEVICE)
    trained_model.eval()
    
    test_dataloader = prepare_test_dataloader(file_name, model_id, project_name)
    predictions = do_inference(trained_model, test_dataloader)
    
    return predictions


def clean_hf_cache(model_id=None):
    """
    Cleans the cache of files downloaded from Hugging Face.
    
    Args:
        model_id (str, optional): Specific model ID to clean.
                                  If not specified, cleans the entire cache.
    """
    # Use the correct cache path
    cache_path = os.path.join(str(Path.home()), '.cache', 'huggingface', 'hub')
    
    if not os.path.exists(cache_path):
        print(f"Cache directory not found at: {cache_path}")
        return
        
    if model_id:
        # If a model is specified, look for its specific directory
        model_path = os.path.join(cache_path, 'models--' + model_id.replace('/', '--'))
        if os.path.exists(model_path):
            print(f"Deleting cache for model: {model_id}")
            try:
                shutil.rmtree(model_path)
                print(f"Cache successfully deleted for: {model_id}")
            except Exception as e:
                print(f"Error deleting cache: {e}")
        else:
            print(f"Cache not found for model: {model_id}")
    else:
        # Clean the entire cache
        print(f"Deleting all Hugging Face cache at: {cache_path}")
        try:
            for item in os.listdir(cache_path):
                item_path = os.path.join(cache_path, item)
                if os.path.isdir(item_path):
                    shutil.rmtree(item_path)
                else:
                    os.remove(item_path)
            print("Cache completely deleted")
        except Exception as e:
            print(f"Error deleting cache: {e}")


def clean_gpu_memory():
    """
    Clean GPU memory by releasing cache memory and unused tensors
    """
    global MODEL

    del MODEL

    # Release PyTorch cache memory
    torch.cuda.empty_cache()
    
    # Release tensor memory
    if torch.cuda.is_available():
        # Get the current device
        device = torch.cuda.current_device()
        
        # Synchronize the device to ensure all operations are complete
        torch.cuda.synchronize(device)
        
        # Force garbage collection
        gc.collect()
        
        # Release all tensors assigned to the device
        torch.cuda.empty_cache()
        
        # Reset all CUDA devices
        torch.cuda.reset_peak_memory_stats(device)
        torch.cuda.reset_accumulated_memory_stats(device)


def local_model_inference(models):
    maes = []
    mdaes = []
    standard_accuracies = []
    for i, model in enumerate(models):
        train_project = model['train']
        test_project = model['test']
        model_name = f"{train_project}_{test_project}"
        # Open the file in results to upload the model
        with open(f"./results/{train_project}_{test_project}.txt", "r") as f:
            model_results = f.read()
            mae, mdae, training_time, epochs, batch_size = (
                float(re.search(r"MAE:\s*([\d.]+)", model_results).group(1)),
                float(re.search(r"MdAE:\s*([\d.]+)", model_results).group(1)),
                float(re.search(r"training time:\s*([\d.]+)", model_results).group(1)),
                int(re.search(r"Epochs:\s*(\d+)", model_results).group(1)),
                int(re.search(r"batch size:\s*(\d+)", model_results).group(1))
            )

        mae, mdae, sa = main(f"./models/{model_name}_epo_{epochs}", train_project)
        mae = round(mae, 2)
        mdae = round(mdae, 2)
        sa = round(sa, 2)
        maes.append(mae)
        mdaes.append(mdae)
        standard_accuracies.append(sa)

        print(f"Model: {model_name}, MAE: {mae}, MdAE: {mdae}, SA: {sa}")

        clean_gpu_memory()
    return maes, mdaes, standard_accuracies


def create_df(maes, mdaes, saes, models):
    data = [
        {
            'Project': train_proj,
            'MAE': mae,
            'MdAE': mdae,
            'SA': sa,
        }
        if (train_proj := model['train']) == (test_proj := model['test'])
        else {
            'Train': train_proj,
            'Test': test_proj,
            'MAE': mae,
            'MdAE': mdae,
            'SA': sa,
        }
        for model, mae, mdae, sa in zip(models, maes, mdaes, saes)
    ]
    return pd.DataFrame(data)

In [2]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.set_per_process_memory_fraction(0.8)

PROJECTS = [
    [
        {'train': 'appceleratorstudio', 'test': 'appceleratorstudio'},
        {'train': 'aptanastudio', 'test': 'aptanastudio'},
        {'train': 'bamboo', 'test': 'bamboo'},
        {'train': 'clover', 'test': 'clover'},
        {'train': 'datamanagement', 'test': 'datamanagement'},
        {'train': 'duracloud', 'test': 'duracloud'},
        {'train': 'jirasoftware', 'test': 'jirasoftware'},
        {'train': 'mesos', 'test': 'mesos'},
        {'train': 'moodle', 'test': 'moodle'},
        {'train': 'mule', 'test': 'mule'},
        {'train': 'mulestudio', 'test': 'mulestudio'},
        {'train': 'springxd', 'test': 'springxd'},
        {'train': 'talenddataquality', 'test': 'talenddataquality'},
        {'train': 'talendesb', 'test': 'talendesb'},
        {'train': 'titanium', 'test': 'titanium'},
        {'train': 'usergrid', 'test': 'usergrid'},
    ],
    [
        {'train': 'mesos', 'test': 'usergrid'},
        {'train': 'usergrid', 'test': 'mesos'},
        {'train': 'appceleratorstudio', 'test': 'aptanastudio'},
        {'train': 'appceleratorstudio', 'test': 'titanium'},
        {'train': 'titanium', 'test': 'appceleratorstudio'},
        {'train': 'aptanastudio', 'test': 'titanium'},
        {'train': 'mule', 'test': 'mulestudio'},
        {'train': 'mulestudio', 'test': 'mule'}
    ],
    [
        {'train': 'clover', 'test': 'usergrid'},
        {'train': 'talendesb', 'test': 'mesos'},
        {'train': 'talenddataquality', 'test': 'aptanastudio'},
        {'train': 'mule', 'test': 'titanium'},
        {'train': 'talenddataquality', 'test': 'appceleratorstudio'},
        {'train': 'mulestudio', 'test': 'titanium'},
        {'train': 'appceleratorstudio', 'test': 'mulestudio'},
        {'train': 'appceleratorstudio', 'test': 'mule'}
    ]
]

MODELS = {
    "#0": "Llama3.2",
    "#2": "Llama3.2+SPWordLevel",
    "#6": "Llama3.2+SPWordPiece",
    "#7": "Llama3.2+SPSentencePiece"
}

WITHIN_PROJECTS = PROJECTS[0]
CROSS_PROJECTS = PROJECTS[1:]
ALL_CROSS_PROJECTS = [item for sublist in PROJECTS[1:] for item in sublist]

<a id='within_project'></a>
## Within Projects Models

#### There are two parts under Within Project Model section, follow the link to reach the section:
#### 1. [Training Process Inspection](#within_project_tb)
#### 2. [Model Testing](#within_project_model_testing)

#### Different models are available for cross project estimation as follows: 

#### #0 - Llama3.2 Auto Tokenizer + Llama3.2
#### #2 - Word-level Story Point Tokenizer + Llama3.2
#### #6 - WordPiece Story Point Tokenizer + Llama3.2
#### #7 - SentencePiece Story Point Tokenizer + Llama3.2 

<a id='within_project_model_testing'></a>
### Model Testing

##### Run the cell below to do inference on all testing datasets using all the uploaded models on Hugging Face

In [3]:
llama3_within_df = pd.DataFrame({'Project': [item['train'] for item in WITHIN_PROJECTS]}, dtype='float64')
llama3_within_df_mdae = llama3_within_df.copy()
llama3_within_df_sa = llama3_within_df.copy()

for model in MODELS:
    maes = []
    mdmaes = []
    standard_accuracies = []
    
    for project in WITHIN_PROJECTS:
        print(f"Running inference for project: {project['train']}")
        mae, mdae, sa = main(model, project['train'])

        mae = round(mae, 2)
        mdae = round(mdae, 2)
        sa = round(sa, 2)
        maes.append(mae)
        mdmaes.append(mdae)
        standard_accuracies.append(sa)
        print(f"MAE for project {project['train']}: {mae}")
        print(f"MDAE for project {project['train']}: {mdae}")
        print(f"Standard Accuracy for project {project['train']}: {sa}")
        
        model_id = f"DEVCamiloSepulveda/{model[1:]}-LLAMA3SP-{project['train']}"
        # clean_hf_cache(model_id)
        clean_gpu_memory()
    llama3_within_df[MODELS[model]] = maes
    llama3_within_df[MODELS[model]] = llama3_within_df[MODELS[model]].apply(lambda x: round(x, 2))
    llama3_within_df_mdae[MODELS[model]] = mdmaes
    llama3_within_df_mdae[MODELS[model]] = llama3_within_df_mdae[MODELS[model]].apply(lambda x: round(x, 2))
    llama3_within_df_sa[MODELS[model]] = standard_accuracies
    llama3_within_df_sa[MODELS[model]] = llama3_within_df_sa[MODELS[model]].apply(lambda x: round(x, 2))

print("All projects processed")

llama3_within_df

C:\Users\Camilo\AppData\Local\Temp\ipykernel_4464\1050825071.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  llama3_within_df = pd.DataFrame({'Project': [item['train'] for item in WITHIN_PROJECTS]}, dtype='float64')


Running inference for project: appceleratorstudio
within project inference using model #0 for project appceleratorstudio
Loading model from Hugging Face...


c:\Users\Camilo\AppData\Local\Programs\Python\Python39\lib\site-packages\accelerate\utils\modeling.py:841: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  512
using llama3 tokenizer!


C:\Users\Camilo\AppData\Local\Temp\ipykernel_4464\2661113312.py:245: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_seq = torch.tensor(tokens_test['input_ids'])


...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  405
using sentencepiece tokenizer!
MAE for project titanium: 2.32
MDAE for project titanium: 1.57
Standard Accuracy for project titanium: 51.29
Running inference for project: usergrid
within project inference using model #7 for project usergrid
Loading model from Hugging Face...


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  86
using sentencepiece tokenizer!
MAE for project usergrid: 1.37
MDAE for project usergrid: 1.25
Standard Accuracy for project usergrid: 44.14
All projects processed


,Project,Llama3.2,Llama3.2+SPWordLevel,Llama3.2+SPWordPiece,Llama3.2+SPSentencePiece
0,appceleratorstudio,1.65,1.50,1.67,1.64
1,aptanastudio,3.74,3.76,4.04,3.74
2,bamboo,1.09,1.43,1.17,0.97
3,clover,4.08,4.90,3.84,3.82
4,datamanagement,6.45,16.79,6.78,7.17
5,duracloud,1.05,1.37,1.29,1.10
6,jirasoftware,2.05,2.32,2.13,1.90
7,mesos,1.38,1.51,1.19,1.29
8,moodle,11.61,11.64,13.81,11.22
9,mule,2.65,2.69,2.80,2.51


##### Optional: Save the results

You can save the results of the DataFrame to a CSV file for further analysis or record-keeping. Run the following script to save the `llama3_within_df` DataFrame to a CSV file.

In [13]:
# Save the DataFrame to a CSV file
llama3_within_df.to_csv('./data_model_analysis/Llama3_within_results.csv', index=False)#

print("CSV file 'Llama3_within_results.csv' created successfully.")

CSV file 'Llama3_within_results.csv' created successfully.


In [4]:
llama3_within_df_mdae.to_csv('./data_model_analysis/Llama3_within_results_mdae.csv', index=False)
llama3_within_df_sa.to_csv('./data_model_analysis/Llama3_within_results_sa.csv', index=False)

print("CSV files 'Llama3_within_results_mdae.csv' and 'Llama3_within_results_sa.csv' created successfully.")

##### Run the cell below to do inference on testing dataset using **local** trained model on all within projects

In [3]:
print("Loading models from local on Within projects inference...")
        
within_maes, within_mdaes, within_saes = local_model_inference(WITHIN_PROJECTS)
within_df = create_df(within_maes, within_mdaes, within_saes, WITHIN_PROJECTS)

within_df

Loading models from local on Within projects inference...
within project inference using model ./models/appceleratorstudio_appceleratorstudio_epo_1 for project appceleratorstudio
Loading model from local...


c:\Users\Camilo\AppData\Local\Programs\Python\Python39\lib\site-packages\accelerate\utils\modeling.py:841: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  512
using llama3 tokenizer!


C:\Users\Camilo\AppData\Local\Temp\ipykernel_21024\2126901288.py:245: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_seq = torch.tensor(tokens_test['input_ids'])


Model: appceleratorstudio_appceleratorstudio, MAE: 1.59, MdAE: 1.34, SA: 65.28
within project inference using model ./models/aptanastudio_aptanastudio_epo_12 for project aptanastudio
Loading model from local...


...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  405
using llama3 tokenizer!
Model: titanium_titanium, MAE: 2.6, MdAE: 1.98, SA: 45.25
within project inference using model ./models/usergrid_usergrid_epo_14 for project usergrid
Loading model from local...


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  86
using llama3 tokenizer!
Model: usergrid_usergrid, MAE: 1.47, MdAE: 1.06, SA: 39.79


,Project,MAE,MdAE,SA
0,appceleratorstudio,1.59,1.34,65.28
1,aptanastudio,3.89,3.10,42.05
2,bamboo,1.11,1.01,19.50
3,clover,3.99,2.61,10.45
4,datamanagement,5.84,2.94,24.53
5,duracloud,1.01,0.70,22.61
6,jirasoftware,2.38,1.82,24.54
7,mesos,1.27,1.03,43.57
8,moodle,13.90,12.57,-109.09
9,mule,2.76,2.55,33.16


##### Optional: Save the results

You can save the results of the DataFrame to a CSV file for further analysis or record-keeping. Run the following script to save the `within_df` DataFrame to a CSV file.

In [4]:
# Save the DataFrame to a CSV file
within_df.to_csv('./data_model_analysis/Llama3SP_within_results.csv', index=False)

print("CSV file 'within_project_results.csv' created successfully.")

CSV file 'within_project_results.csv' created successfully.


<a id='cross_project'></a>
## Cross Projects Models

#### There are two parts under Cross Project Model section, follow the link to reach the section:
#### 1. [Training Process Inspection](#cross_project_tb)
#### 2. [Model Testing](#cross_project_model_testing)

##### Different models are available for cross project estimation as follows:

### Cross project - within repository models
#### #00 - Llama3.2 Auto Tokenizer
#### #22 - Word-level Story Point Tokenizer + Llama3.2
#### #66 - WordPiece Story Point Tokenizer + Llama3.2
#### #77 - SentencePiece Story Point Tokenizer + Llama3.2  

### Cross project - cross repository models
#### #000 - Llama3.2 Auto Tokenizer
#### #222 - Word-level Story Point Tokenizer + Llama3.2
#### #666 - WordPiece Story Point Tokenizer + Llama3.2
#### #777 - SentencePiece Story Point Tokenizer + Llama3.2 

<a id='cross_project_model_testing'></a>
### Model Testing

##### Run the cell below to do inference on all testing datasets using all the uploaded models on Hugging Face

In [5]:
llama3_cross_df = pd.DataFrame(
    {
        'Train': [item['train'] for item in ALL_CROSS_PROJECTS],
        'Test': [item['test'] for item in ALL_CROSS_PROJECTS]
    }
    , dtype='float64'
)
llama3_cross_df_mdae = llama3_cross_df.copy()
llama3_cross_df_sa = llama3_cross_df.copy()

for model in MODELS:
    maes = []
    mdmaes = []
    standard_accuracies = []

    for i, projects in enumerate(CROSS_PROJECTS):
        for project in projects:
            model_id = model
            caracter = model[1:]
            caracter = caracter * (i + 2)
            model_id = f"#{caracter}"

            print(f"Running inference for project trained on {project['train']} and tested on {project['test']}")
            mae, mdae, sa = main(model_id, f"{project['train']}_{project['test']}" )
            
            mae = round(mae, 2)
            mdae = round(mdae, 2)
            sa = round(sa, 2)
            maes.append(mae)
            mdmaes.append(mdae)
            standard_accuracies.append(sa)
            print(f"MAE for project {project['train']}: {mae}")
            print(f"MDAE for project {project['train']}: {mdae}")
            print(f"Standard Accuracy for project {project['train']}: {sa}")

            hf_model_id = f"DEVCamiloSepulveda/{model[1:]}-LLAMA3SP-{project['train']}"
            # clean_hf_cache(hf_model_id)
            clean_gpu_memory()
    llama3_cross_df[MODELS[model]] = maes
    llama3_cross_df[MODELS[model]] = llama3_cross_df[MODELS[model]].apply(lambda x: round(x, 2))
    llama3_cross_df_mdae[MODELS[model]] = mdmaes
    llama3_cross_df_mdae[MODELS[model]] = llama3_cross_df_mdae[MODELS[model]].apply(lambda x: round(x, 2))
    llama3_cross_df_sa[MODELS[model]] = standard_accuracies
    llama3_cross_df_sa[MODELS[model]] = llama3_cross_df_sa[MODELS[model]].apply(lambda x: round(x, 2))


print("All projects processed")

llama3_cross_df

C:\Users\Camilo\AppData\Local\Temp\ipykernel_21024\3632030941.py:1: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  llama3_cross_df = pd.DataFrame(


Running inference for project trained on mesos and tested on usergrid
cross project inference using model #00 trained on mesos for project usergrid
Loading model from Hugging Face...


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


using llama3 tokenizer!


C:\Users\Camilo\AppData\Local\Temp\ipykernel_21024\2126901288.py:259: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_seq = torch.tensor(tokens_test['input_ids'])


...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


using sentencepiece tokenizer!
MAE for project appceleratorstudio: 3.53
MDAE for project appceleratorstudio: 2.46
Standard Accuracy for project appceleratorstudio: 40.16
Running inference for project trained on appceleratorstudio and tested on mule
cross project inference using model #777 trained on appceleratorstudio for project mule
Loading model from Hugging Face...


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


using sentencepiece tokenizer!
MAE for project appceleratorstudio: 2.77
MDAE for project appceleratorstudio: 2.44
Standard Accuracy for project appceleratorstudio: 39.48
All projects processed


,Train,Test,Llama3.2,Llama3.2+SPWordLevel,Llama3.2+SPWordPiece,Llama3.2+SPSentencePiece
0,mesos,usergrid,1.34,1.40,1.38,1.05
1,usergrid,mesos,1.74,1.73,1.60,1.56
2,appceleratorstudio,aptanastudio,4.36,5.23,4.28,4.31
3,appceleratorstudio,titanium,3.36,3.77,3.41,3.30
4,titanium,appceleratorstudio,2.55,2.64,2.36,2.36
5,aptanastudio,titanium,3.78,3.64,4.11,3.78
6,mule,mulestudio,3.57,4.07,3.43,3.65
7,mulestudio,mule,2.98,3.07,3.09,3.31
8,clover,usergrid,2.03,2.53,2.07,1.48
9,talendesb,mesos,1.52,1.95,1.60,1.66


##### Optional: Save the results

You can save the results of the DataFrame to a CSV file for further analysis or record-keeping. Run the following script to save the `llama3_cross_df` DataFrame to a CSV file.

In [6]:
# Save the DataFrtaFrame to a CSV file
llama3_cross_df.to_csv('./data_model_analysis/Llama3_cross_results.csv', index=False)

print("CSV file 'Llama3_cross_results.csv' created successfully.")

CSV file 'Llama3_cross_results.csv' created successfully.


In [6]:
llama3_cross_df_mdae.to_csv('./data_model_analysis/Llama3_cross_results_mdae.csv', index=False)
llama3_cross_df_sa.to_csv('./data_model_analysis/Llama3_cross_results_sa.csv', index=False)

print("CSV file 'Llama3_cross_results_mdae.csv' and 'Llama3_cross_results_sa.csv' created successfully.")

CSV file 'Llama3_cross_results_mdae.csv' and 'Llama3_cross_results_sa.csv' created successfully.


##### Run the cell below to do inference on testing dataset using **local** trained model on all cross projects

In [7]:
print("Loading models from local on Cross projects inference...")

cross_projects = [item for sublist in PROJECTS[1:] for item in sublist]
cross_maes, cross_mdaes, cross_saes = local_model_inference(cross_projects)
cross_df = create_df(cross_maes, cross_mdaes, cross_saes, cross_projects)

cross_df

Loading models from local on Cross projects inference...
within project inference using model ./models/mesos_usergrid_epo_6 for project mesos
Loading model from local...


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  302
using llama3 tokenizer!


C:\Users\Camilo\AppData\Local\Temp\ipykernel_21024\2126901288.py:245: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_seq = torch.tensor(tokens_test['input_ids'])


Model: mesos_usergrid, MAE: 1.37, MdAE: 1.06, SA: 39.19
within project inference using model ./models/usergrid_mesos_epo_4 for project usergrid
Loading model from local...


...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  512
using llama3 tokenizer!
Model: appceleratorstudio_mulestudio, MAE: 1.74, MdAE: 1.54, SA: 61.97
within project inference using model ./models/appceleratorstudio_mule_epo_2 for project appceleratorstudio
Loading model from local...


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batch Size:  512
using llama3 tokenizer!
Model: appceleratorstudio_mule, MAE: 1.91, MdAE: 1.71, SA: 58.32


,Train,Test,MAE,MdAE,SA
0,mesos,usergrid,1.37,1.06,39.19
1,usergrid,mesos,1.45,1.20,40.64
2,appceleratorstudio,aptanastudio,1.53,1.23,66.62
3,appceleratorstudio,titanium,1.70,1.45,62.93
4,titanium,appceleratorstudio,2.65,2.19,44.29
5,aptanastudio,titanium,3.48,2.80,48.17
6,mule,mulestudio,2.73,2.52,33.71
7,mulestudio,mule,3.87,3.04,39.99
8,clover,usergrid,4.17,2.55,6.60
9,talendesb,mesos,1.07,0.87,30.58


##### Optional: Save the results

You can save the results of the DataFrame to a CSV file for further analysis or record-keeping. Run the following script to save the `cross_df` DataFrame to a CSV file.

In [8]:
# Save the DataFrtaFrame to a CSV file
cross_df.to_csv('./data_model_analysis/Llama3SP_cross_results.csv', index=False)

print("CSV file 'Llama3SP_cross_results.csv' created successfully.")

CSV file 'Llama3SP_cross_results.csv' created successfully.
